In [1]:
# For computer vision, I am using torchvision
"""
torchvision.datasets
torchvision.models
torch.utils.data.Dataset
torch.utils.data.DataLoader
"""

'\ntorchvision.datasets\ntorchvision.models\ntorch.utils.data.Dataset\ntorch.utils.data.DataLoader\n'

In [ ]:
import torch
from torch import nn    # neural network
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

torch.__version__, torchvision.__version__

In [ ]:
# Getting a dataset ( FashionMNIST)

train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor(), target_transform=None)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor(), target_transform=None)

In [ ]:
len(train_data), len(test_data)

In [ ]:
train_data.classes

In [ ]:
#Converting classes to dictionary

train_data.class_to_idx

In [ ]:
image, label = train_data[0]

image.shape, label

In [ ]:
classes = train_data.classes

classes[label]

In [ ]:
image, label = train_data[0]
print(f"Image shape : {image.shape}")
plt.figure(figsize=(3,2))
plt.imshow(image.permute(1,2,0))
plt.title(classes[label])

In [ ]:
plt.figure(figsize=(3,2))
plt.imshow(image.permute(1,2,0), cmap='gray')

In [ ]:
torch.manual_seed(42)
fig = plt.figure(figsize=(10,7))
rows, cols = 4, 4
for i in range(1, rows*cols + 1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.permute(1,2,0), cmap='gray')
    plt.title(classes[label])
    plt.axis(False)

In [ ]:
# Preparing DataLoader

from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

train_dataloader, test_dataloader

In [ ]:
print(f"Dataloaders : {train_dataloader, test_dataloader}")
print(f"Length of train_dataloader : {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test_dataloader : {len(test_dataloader)} batches of {BATCH_SIZE}")

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
test_features_batch, test_labels_batch = next(iter(test_dataloader))
test_features_batch.shape

In [ ]:
#torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]

plt.figure(figsize=(5,3))
plt.imshow(img.squeeze())
plt.title(classes[label])
plt.axis(False)

In [ ]:
# Creating a baseline model

flatten_model = nn.Flatten()

x = train_features_batch[0]
# x.shape = torch.Size([1, 28, 28])

output = flatten_model(x)

output.shape

In [ ]:
class FashionMNISTModel(nn.Module):
    def __init__(self, input_shape : int, hidden_units : int, output_shape : int):
        super().__init__()
        self.layerStack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.Linear(in_features=hidden_units, out_features=output_shape)
        )
    def forward(self, x : torch.Tensor) -> torch.Tensor:
        return self.layerStack(x)

In [ ]:
torch.manual_seed(42)

modelv0 = FashionMNISTModel(input_shape=784,hidden_units=10, output_shape=len(classes))

modelv0

In [ ]:
# passing a dummy sample

dummy_x = torch.rand([1,1,28,28])

modelv0(dummy_x)

In [ ]:
# Setting up loss function and optimizer

from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=modelv0.parameters(), lr = 0.03)

In [ ]:
# Setting up a timer function
from timeit import default_timer as timer
def print_train_time(start:float, end:float, device:torch.device=None):
    total_time = end-start
    print(f"Train time  : { total_time:.3f} seconds")
    return total_time

In [ ]:
# Testing the timer function
start_time = timer()
end_time = timer()
print_train_time(start_time, end_time)

In [ ]:
list = [([[[1,2,3],[4,5,6]]],9),([[[1,2,3],[5,8,9]]],1),([[[5,3,2],[6,5,3]]],5)]

for batch, (x, y) in enumerate(list):
    print("This is index ", batch)
    print("This is x ", x)
    print("This is y ", y)

In [ ]:
from tqdm.auto import tqdm

torch.manual_seed(42)

epochs = 3

train_time_start = timer()

for epoch in tqdm(range(epochs)):
    print(f"Epoch : {epoch}")
    train_loss = 0
    for batch, (img,label) in enumerate(train_dataloader):
        modelv0.train()
        train_preds = modelv0(img)
        loss = loss_fn(train_preds, label)
        train_loss+=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 400 == 0:
            print(f"Looked at {batch * len(img)}/{len(train_dataloader.dataset)} samples.")

    train_loss /= len(train_dataloader)
    test_loss = 0
    test_acc = 0
    modelv0.eval()
    with torch.inference_mode():
        for img, label in test_dataloader:
            test_pred = modelv0(img)
            test_loss += loss_fn(test_pred, label)

            test_acc += accuracy_fn(y_true=label, y_pred=test_pred.argmax(dim=1))

        test_loss /= len(test_dataloader)
        test_acc /= len(test_dataloader)

        print(f"Train loss : {train_loss:.4f}, test loss {test_loss:4f}, test_acc : {test_acc:.4f}")
train_time_end = timer()

total_train_time_modelv0 = print_train_time(start=train_time_start, end=train_time_end)

In [ ]:
torch.manual_seed(42)
def eval_model(model : torch.nn.Module, data_loader : torch.utils.data.DataLoader, loss_fn : torch.nn.Module, accuracy_fn):
    loss, acc = 0, 0
    modelv0.eval()
    with torch.inference_mode():
        for X,y in data_loader:
            test_pred = model(X)
            loss += loss_fn(test_pred, y)
            acc += accuracy_fn(y_true=y, y_pred = test_pred.argmax(dim=1))

        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model" : model.__class__.__name__,
            "loss" : loss.item(),
            "acc" : acc
            }

modelv0_results = eval_model(model=modelv0, data_loader=test_dataloader,loss_fn=loss_fn, accuracy_fn=accuracy_fn)


modelv0_results

In [ ]:
## Setting up device agnostic code

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
# Another model to improve the performance

class FashionMNISTModel1(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_units):
        super().__init__()

        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape)
        )

    def forward(self, x : torch.Tensor) -> torch.Tensor:
        return self.layer_stack(x)

In [ ]:
modelv1 = FashionMNISTModel1(input_shape=784, output_shape=len(classes), hidden_units=10).to(device)

modelv1

In [ ]:
from helper_functions import accuracy_fn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=modelv1.parameters(), lr = 0.1)

In [ ]:
# Setting up training step function

def train_step(model : torch.nn.Module, data_loader : torch.utils.data.DataLoader, loss_fn : torch.nn.Module, optimizer : torch.optim.Optimizer, accuracy_fn, device : torch.device = device):

    train_loss = 0
    acc = 0
    model.train()
    for batch, (img,label) in enumerate(data_loader):
        train_preds = model(img)
        loss = loss_fn(train_preds, label)
        train_loss+=loss
        acc+= accuracy_fn(y_true=label, y_pred=train_preds.argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 400 == 0:
            print(f"Looked at {batch * len(img)}/{len(train_dataloader.dataset)} samples.")

    train_loss /= len(data_loader)
    acc /= len(data_loader)

    print(f"Train loss : {train_loss:.4f}, train acc : {acc:2f}%")

In [ ]:
# Setting up a function for testing loop
def test_step(model : torch.nn.Module, data_loader : torch.utils.data.DataLoader, loss_fn : torch.nn.Module, accuracy_fn, device : torch.device = device):

    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for x,y in data_loader:
            test_preds = model(x)
            test_loss += loss_fn(test_preds, y)
            test_acc += accuracy_fn(y_true=y, y_pred=test_preds.argmax(dim=1))

        test_loss /= len(data_loader)
        test_acc /= len(data_loader)

        print(f"Test loss : {test_loss:.3f}, test acc : {test_acc:.2f}%")


In [ ]:
# Training and testing loop


from timeit import default_timer as timer

train_time_start = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
    print(f"Epoch : {epoch}")

    train_step(model=modelv1, data_loader=train_dataloader, loss_fn=loss_fn, optimizer=optimizer, accuracy_fn=accuracy_fn, device=device)

    test_step(model=modelv1, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_fn=accuracy_fn, device=device)

train_time_end = timer()

total_train_time_modelv1 = print_train_time(start=train_time_start, end=train_time_end)


In [ ]:
modelv0_results, total_train_time_modelv0

In [ ]:
modelv1_results = eval_model(model=modelv1, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_fn=accuracy_fn)

modelv1_results, total_train_time_modelv1